In [ ]:
import pandas as pd
import numpy as np

# Load behaviors data
behaviors_cols = ["impression_id", "user_id", "time", "history", "impressions"]
behaviors_df = pd.read_csv("train_data/behaviors.tsv", sep="\t", header=None, names=behaviors_cols)

# Check for duplicates
duplicate_impressions = behaviors_df.duplicated(subset=["impression_id"]).sum()
print(f"Duplicate impression IDs: {duplicate_impressions}")

# Check missing values
print("\nMissing values in behaviors data:")
print(behaviors_df.isnull().sum())

# Parse impressions into expanded format
print("\nParsing impression logs...")
impressions_expanded = []
for _, row in behaviors_df.iterrows():
    impression_items = row['impressions'].split()
    for item in impression_items:
        parts = item.split('-')
        if len(parts) == 2:
            news_id, click = parts
            impressions_expanded.append({
                'impression_id': row['impression_id'],
                'user_id': row['user_id'],
                'news_id': news_id,
                'clicked': int(click)
            })

impressions_df = pd.DataFrame(impressions_expanded)
print(f"Expanded to {len(impressions_df)} impression records")

# Check for invalid news IDs (not in news_df)
news_ids = set(news_df["newsID"])
invalid_news_ids = impressions_df[~impressions_df["news_id"].isin(news_ids)]
print(f"Impression records with invalid news IDs: {len(invalid_news_ids)}")

# Check click distribution
clicks = impressions_df["clicked"].sum()
total = len(impressions_df)
print(f"\nOverall CTR: {clicks/total:.4f} ({clicks} clicks out of {total} impressions)")

# Check for articles with too few impressions (unreliable CTR)
article_impressions = impressions_df.groupby("news_id").size()
low_impression_articles = (article_impressions < 5).sum()
print(f"Articles with fewer than 5 impressions: {low_impression_articles}")

# Check for extreme CTRs (potential data issues)
article_ctrs = impressions_df.groupby("news_id")["clicked"].mean()
suspicious_ctrs = ((article_ctrs == 0) | (article_ctrs > 0.7)).sum()
print(f"Articles with suspicious CTRs (0 or >70%): {suspicious_ctrs}")

# Potential adjustments based on exploration:
# 1. Filter out articles with too few impressions (e.g., < 10)
# 2. Investigate articles with extreme CTRs
# 3. Set minimum threshold for impression count when calculating CTR